# PiPP autoregressive prepositional effects

In [1]:
__author__ = "Christopher Potts"

## Set-up

In [2]:
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
from minicons import scorer
import os
import pandas as pd
import re

import utils

In [3]:
plt.style.use("pipp.mplstyle")

## Items

In [4]:
def item(ex):
    assert "*" in ex
    target = re.search(r"\*(\w+)", ex).group(1)
    data = {}
    data["though/–gap"] = (ex.replace("*", ""), target)
    data["although/–gap"] = (ex.replace("*", "").replace('though', 'although'), target)
    data["though/+gap"] = (re.sub("\s*\*.*", ".", ex), ".")
    data["although/+gap"] = (re.sub("\s*\*.*", ".", ex).replace('though', 'although'), ".")
    return data

## Materials

In [5]:
with open("materials-autoregressive-prepeffects.txt") as f:
    materials = f.read().splitlines()

In [6]:
for ex in materials:
    print("=" * 70)
    print(item(ex))

{'though/–gap': ('She liked the food spicy though it was rarely served that way.', 'rarely'), 'although/–gap': ('She liked the food spicy although it was rarely served that way.', 'rarely'), 'though/+gap': ('She liked the food spicy though it was.', '.'), 'although/+gap': ('She liked the food spicy although it was.', '.')}
{'though/–gap': ('They considered the idea scary though it was boring.', 'boring'), 'although/–gap': ('They considered the idea scary although it was boring.', 'boring'), 'though/+gap': ('They considered the idea scary though it was.', '.'), 'although/+gap': ('They considered the idea scary although it was.', '.')}
{'though/–gap': ('The restaurant served the soup cold though it was usually served hot.', 'usually'), 'although/–gap': ('The restaurant served the soup cold although it was usually served hot.', 'usually'), 'though/+gap': ('The restaurant served the soup cold though it was.', '.'), 'although/+gap': ('The restaurant served the soup cold although it was.', '

## Experiments

In [7]:
def run_item(ex, item_num, model):
    data = []
    conds = item(ex)
    for typ, (text, target) in conds.items():
        response = {}
        response['fulltext'] = text
        response['prompt'] = text
        toks_with_logprobs = model.token_score([text], rank=False)[0]
        toks, logprobs = zip(*toks_with_logprobs)
        response['prompt_tokens'] = list(toks)
        response['prompt_scores'] = list(logprobs)
        inds = [i for i, tok in enumerate(toks) if tok.strip() == target]
        ti = inds[0]
        surprisal = convert_to_surprisal(logprobs[ti])
        response['condition'] = typ
        response['target_surprisal'] = surprisal
        response['item_num'] = item_num
        data.append(response)
    return data

def convert_to_surprisal(x):
    return -(x / np.log(2))

In [8]:
def run_experiment(materials, model_name):
    data = []
    model = scorer.IncrementalLMScorer(model_name)
    for item_num, m in enumerate(materials, start=1):
        data += run_item(m, item_num, model)
    model_nickname = model_name.split("/")[-1]
    output_filename = f"results/autoregressive-pp-results-{model_nickname}.json"
    with open(output_filename, "wt") as f:
        json.dump(data, f, indent=4)

### Models

In [9]:
all_model_names = [
    "EleutherAI/pythia-70m-deduped",
    "EleutherAI/pythia-160m-deduped",
    "EleutherAI/pythia-410m-deduped",
    "EleutherAI/pythia-1b-deduped",
    "EleutherAI/pythia-1.4b-deduped",
    "EleutherAI/pythia-2.8b-deduped",
    "EleutherAI/pythia-6.9b-deduped"
    "EleutherAI/pythia-12b-deduped"
]

for model_name in all_model_names:
    run_experiment(materials, model_name)

## Visualization

In [10]:
order = ['though/+gap', 'although/+gap', 'though/–gap',  'although/–gap'][::-1]

for results_filename in glob.glob(os.path.join("results", "autoregressive-pp-results*.json")):
    utils.mean_plot(results_filename, order, xlim=18)